In [1]:
import pandas as pd
import re
!pip install hazm
from hazm import word_tokenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, TimeDistributed, Flatten
from keras import Model
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np

In [30]:
!gdown --id 1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg
To: /content/hamshahri.rar
100% 873M/873M [00:03<00:00, 274MB/s]


In [31]:
!unrar x /content/hamshahri.rar -d /content


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/hamshahri.rar


Would you like to replace the existing file /content/hamshahriold/CLEF/images/ar.JPG
  1114 bytes, modified on 2008-05-04 12:51
with a new one
  1114 bytes, modified on 2008-05-04 12:51

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit 
User break

User break


In [40]:
!unzip /content/hamshahriold/Corpus/Hamshahri-Corpus.zip -d /content/DataSet
!unzip /content/hamshahriold/Corpus/PersianStopWords.zip -d /content/StopWords

Archive:  /content/hamshahriold/Corpus/Hamshahri-Corpus.zip
replace /content/DataSet/Hamshahri-Corpus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/DataSet/Hamshahri-Corpus.txt  
Archive:  /content/hamshahriold/Corpus/PersianStopWords.zip
replace /content/StopWords/PersianStopWords.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/StopWords/PersianStopWords.txt  


In [42]:
with open('/content/DataSet/Hamshahri-Corpus.txt') as f:
    dataset = f.readlines ()
    
for i in range (10) :
  print (dataset[i])

.DID	1S1

.Date	75\04\02

.Cat	adabh

جاودانگي در زندگي گروهي از طريق هنر 

نگاهي به نمايشگاه آثار هنري احمد طباطبايي 

موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين 

انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر 

گوشه هايي از زندگي خود هستند. 

مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه 

خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها 



In [43]:
with open('/content/StopWords/PersianStopWords.txt') as f:
    stopwords = f.readlines ()

stopwords = [ x.replace ('\n', '').replace ('\ufeff', '') for x in stopwords ]
print (stopwords)
print (len(stopwords))

['ـ', '-', '!', '"', '#', '(', ')', '*', '.', '...', '....', '/', ':', '[', ']', '،', '؛', '؟', '«', '»', '…', 'آاو و و و', 'آخ', 'آخر', 'آخرها', 'آخه', 'آدمهاست', 'آرام', 'آرام آرام', 'آره', 'آزادانه', 'آسان', 'آسيب پذيرند', 'آشكارا', 'آشنايند', 'آمرانه', 'آن', 'آن گاه', 'آن ها', 'آنان', 'آناني', 'آنجا', 'آنچنان', 'آنچنان كه', 'آنچه', 'آنرا', 'آنقدر', 'آنگاه', 'آنها', 'آنهاست', 'آهان', 'آهاي', 'آوه', 'آيا', 'ا', 'اتفاقا', 'اجراست', 'احتمالا', 'احياناً', 'اخيراً', 'از', 'از آن پس', 'از جمله', 'ازاين رو', 'ازش', 'اساساً', 'است', 'اسلامي اند', 'اش', 'اشتباها', 'اصلا', 'اصلاً', 'اصولا', 'اصولاً', 'اغلب', 'افسوس', 'اقل', 'اقليت', 'اكثر', 'اكثرا', 'اكثراً', 'اكثريت', 'اكنون', 'اگر', 'اگر چه', 'اگرچه', 'اگه', 'الا', 'الان', 'البته', 'الهي', 'الي', 'اما', 'امروز', 'امروزه', 'امسال', 'امشب', 'اميدوارم', 'اميدوارند', 'اميدواريم', 'ان شاأالله', 'اند', 'اندكي', 'انگار', 'او', 'اوست', 'اولا', 'اولاً', 'اون', 'اي', 'ايشان', 'اين', 'اين جوري', 'اين قدر', 'اين گونه', 'اينان', 'اينجا', 'اينجاست', 'اين

In [44]:
Dataset = []
Cats = []
temp = ""

for x in dataset :

  match1 = re.findall("\.DID", x)
  match2 = re.findall("\.Date", x)
  match3 = re.findall("\.Cat", x)

  if match1 or match2 :
    continue
  elif match3 : 
    x = x.replace ('.Cat', '').replace ('\t', '').replace ('\n', '')
    Cats.append (x)
    if temp != "" :
      Dataset.append (temp)
    temp = ""
  else :
    x = x.replace ('\n', '')
    temp += x

In [45]:
Corpus = []

for index, x in enumerate (Dataset) :
  Corpus.append ([x, Cats [index]])

In [46]:
df = pd.DataFrame(Corpus, columns=['text', 'category'])

In [47]:
df.to_csv("dataset.csv")

In [48]:
for index, txt in enumerate(df["text"]):
  word_tokenized = word_tokenize(txt)
  Clean_Text = ""
  for word in word_tokenized :
    if word not in stopwords :
      Clean_Text += word + " "
      
  df.loc[index].at['text'] = Clean_Text

In [49]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [79]:
df.to_csv("/content/gdrive/MyDrive/save.csv")

In [3]:
df = pd.read_csv ("/content/gdrive/MyDrive/save.csv")

In [4]:
Unique_Cats = []

for cat in df ['category'] :
  if cat not in Unique_Cats :
      Unique_Cats.append (cat)

len (Unique_Cats)

105

In [5]:
Cat_IDs = {word: i for i, word in enumerate(Unique_Cats)}

In [32]:
ID_To_Cat = {idx:l for idx, l in enumerate(Unique_Cats)}

In [6]:
df['category_id'] = df['category'].apply(lambda t: Cat_IDs[t])


In [64]:
df = df.dropna(subset=['text'])
df = df.reset_index(drop=True)

In [7]:
for x in df ['text'] :
  if type (x) == float :
    print (x)

In [17]:
t = Tokenizer()
t.fit_on_texts(df ['text'].tolist())
Word_To_ID = t.word_index
ID_To_Word = {v:k for k, v in Word_To_ID.items()}
wids = [[Word_To_ID[w] for w in text_to_word_sequence(doc)] for doc in df['text'].tolist()]
vocab_size = len(t.word_index) + 1
max_length = 200
X = pad_sequences(wids, maxlen=max_length, padding='post')
y = df['category_id'].tolist()

In [71]:
print (df['text'])

0         جاودانگي زندگي گروهي طريق هنر نگاهي نمايشگاه آ...
1         رويدادهاي هنري جهان نمايشگاه هنر خدمت ديكتاتور...
2         برديوار نگارخانه گالري گلستان نمايشگاه طرح ساخ...
3         بازي جدي بگيريم مطالعه مقدماتي نقش بازي زندگي ...
4         تخته سياه غباري سترده اشاره; رغم گسترش توسعه ر...
                                ...                        
165197    گره كور كشتي شودگروه ورزشي حضور رسمي محمدرضا ط...
165198    نماينده فدراسيون جهاني واليبال ايران نظر شايست...
165199    شكست نامداران تكواندودر پيكارهاي برتر ليگ گروه...
165200    ورزشگاه بزرگ دانشگاه آزاد تهران ساخته گروه ورز...
165201    رئيس فدراسيون پزشكي انتخاب گروه ورزشي مجمع عمو...
Name: text, Length: 165202, dtype: object


In [22]:
# main model
input = Input(shape=(max_length,))
model = Embedding(vocab_size, 128 ,input_length=max_length)(input)
model =  Bidirectional (LSTM (100,return_sequences=True,dropout=0.50),merge_mode='concat')(model)
model = TimeDistributed(Dense(100,activation='relu'))(model)
model = Flatten()(model)
model = Dense(100,activation='relu')(model)
output = Dense(len (Unique_Cats), activation='softmax')(model)
model = Model(input, output)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 200, 128)          66822656  
                                                                 
 bidirectional_6 (Bidirectio  (None, 200, 200)         183200    
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 200, 100)         20100     
 tributed)                                                       
                                                                 
 flatten_1 (Flatten)         (None, 20000)             0         
                                                                 
 dense_8 (Dense)             (None, 100)               2000

In [76]:
print(X.shape)
print (len(y))

(165202, 400)
165202


In [25]:
model.fit(X ,np.array (y) , epochs = 10, verbose = 1)

Epoch 1/10
5163/5163 [==============================] - 287s 56ms/step - loss: 0.9056 - accuracy: 0.7613
Epoch 2/10
5163/5163 [==============================] - 292s 57ms/step - loss: 0.5630 - accuracy: 0.8356
Epoch 3/10
5163/5163 [==============================] - 303s 59ms/step - loss: 0.3521 - accuracy: 0.8929
Epoch 4/10
5163/5163 [==============================] - 308s 60ms/step - loss: 0.2192 - accuracy: 0.9319
Epoch 5/10
 101/5163 [..............................] - ETA: 4:39 - loss: 0.1174 - accuracy: 0.9663

KeyboardInterrupt: ignored

In [26]:
model.save_weights("/content/gdrive/MyDrive/MyModel.h5")

In [ ]:
model.load_weights("/content/gdrive/MyDrive/MyModel.h5")

In [27]:
def preprocess(texts):
  results = []
  stop_words_list = []
  with open('/content/StopWords/PersianStopWords.txt', "rb") as file:
    for line in file:
      stop_words_list.append(line.decode("UTF-8").replace('\r\n', ""))
  for txt in texts:
    word_tokenized = word_tokenize(txt)
    cps = ""
    for word in word_tokenized:
      if word not in stop_words_list:
        cps += word + " "
    results.append(cps)
  return(results)

In [28]:
def prediction(texts):
  result = []
  cleaned_texts = preprocess(texts)
  for cleaned_text in cleaned_texts:
    text_to_sequences = t.texts_to_sequences([cleaned_text])
    pad_text_to_sequences = pad_sequences(text_to_sequences, maxlen=max_length)
    category = model.predict([pad_text_to_sequences])[0]

    category_index = np.where(category == max(category))
    result.append(ID_To_Cat [category_index[0][0]])
  return result

In [33]:
List = ['میرهاشم موسوی در پاسخ به گلایهها و انتقادات بازنشستگان غیرحداقلیبگیر از نحوه افزایش حقوقها در سال جاری اظهار کرد: آنچه ما باید بر اساس قانون به عنوان سازمان تامین اجتماعی انجام میدادیم همان را انجام دادیم و همان مسیری بوده که تا کنون طی کردهایم.  وی افزود: در ماده ۹۶ قانون تامین اجتماعی قید شده است که میزان افزایش حقوق غیرحداقلیبگیران باید بر اساس هزینههای زندگی تعیین و سپس در هیئت وزیران مصوب شود و به اجرا دربیاید. در ماده ۱۱۱ قانون تامین اجتماعی هم آمده است که میزان افزایش حقوق حداقلیبگیران نباید از حداقل دستمزد کارگران که در شورای عالی کار مصوب شده، کمتر باشد.  مدیرعامل سازمان تامین اجتماعی با بیان اینکه ما بر اساس ماده ۹۶ و ۱۱۱ قانون تامین اجتماعی مسیر افزایش حقوقها را طی کرده و مصوبه هیئت امنای تامین اجتماعی را به دولت ارسال کردیم گفت: این تصمیم، تصمیمِ بخشی و سازمانی ما بود و هیئت امنا همین را پذیرفت و به هیئت دولت ارسال کرد اما اقتضائاتی که در هیئت دولت مطرح است، به علاوه نگاه بخشی ما در تامین اجتماعی است و طبیعتا قانونگذار هم این اختیار را بر اساس ماده ۹۶ به هیئت وزیران داده است که این ملاحظات و اقتضائات را درنظر بگیرد.       موسوی ادامه داد: آن پیشنهادی که ما ارائه دادیم، قانون است و اینکه باید اقتضائات در دولت دیده شود هم قانون است؛ هر دو قانون هستند و نمیتوانیم یک سوی قانون را ببینیم و آن سوی قانون را نادیده بگیریم. خب تصمیم نهایی در هیئت دولت با توجه به ملاحظاتی که برای مابقی صندوقها وجود داشت و همچنین دیگر موارد فرابخشی در اقتصاد کلان و بخشهای مختلف، اتخاذ شد. آنجا هیئت وزیران است و همه باید نظر بدهند و تصمیم نهایی اخذ شود.  وی افزود: البته در جلسات کارشناسی مختلفی که ما حضور داشتیم، نظرات خود را ارائه کرده بودیم و تبیین کارشناسی خودمان را داشتیم. در نهایت  تصمیمی که گرفته شد این بود که برای حداقلی بگیران ۵۷.۴ درصد افزایش داشته باشیم. برای سایر سطوح درآمدی نیز دو سطح قائل شدند، نخست حقوق بگیران زیر ۱۰ میلیون تومان که مشمول افزایش ۱۰ درصدی به علاوه ۶۵۰ هزار تومان مبلغ ثابت شدند و دوم حقوقبگیران بالای ۱۰ میلیون تومان که حقوقشان ۱۰ درصد افزایش پیدا کرد.  مدیرعامل سازمان تامین اجتماعی با اشاره به اینکه برای بخشی از "سایر سطوح" که شاید چیزی حدود به ۹۰۰ هزار نفر بازنشسته را شامل میشود، این تصمیم برای افزایش حقوق، محل سوال شد و معترض بودند اظهار کرد: این موضوع در دولت در حال بررسی بود که هیئت تطبیق قوانین مجلس هم ورود پیدا کرد و نظرات خودشان را اعلام کردند.  موسوی درباره نحوه پرداختهای جبرانی به این گروه از بازنشستگان که پیش از این از سوی سرپرست وزارت رفاه وعده داده و در برخی رسانهها منتشر شده است نیز توضیح داد و گفت: اکنون این موضوع در دولت در حال بررسی است. ما نمیتوانیم بگوییم که با چه الگویی میخواهیم این را جبران کنیم. منتظر تصمیم نهایی هستیم که آنچه باید بر اساس قانون انجام دهیم را انجام دهیم و این آمادگی در تامین اجتماعی وجود دارد که وفق قانون هر تصمیمی گرفته شد را عمل کرده و اقدامی که باید برای این عزیزان در پرداخت مستمریها اتفاق بیفتد را انجام دهد.']
prediction (List)

['aeqts']